# Collaborative filtering using the Surprise library

Surprise (http://surpriselib.com/) es una librería Python para crear y analizar sistemas de recomendación colaborativos utilizando datos de preferencia explícitos. Surprise incorpora implementaciones de los algoritmos más utilizados, datasets de prueba ([Movielens](http://grouplens.org/datasets/movielens/), [Jester](http://eigentaste.berkeley.edu/dataset/)), y herramientas para facilitarlla evaluación y comparación de distintos algoritmos y parámetros de los mismos.

## Uso básico de Surprise

El siguiente ejemplo muestra el uso básico de la librería con un dataset con las preferencias de 5 usuarios (A, B, C, D y E) sobre 2 ítems (salvo para el usuario E del que solo se dispone de su voto para el ítem 1). Se emplea un algoritmo basado en vecinos cercanos (el que vimos en teoría en el apartado `Collaborative filtering`).

In [ ]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

# We will use the "KNNWithMeans", a k-NN based algorithm.

from surprise import KNNWithMeans

# Configuration to use item-based cosine similarity

sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

trainingSet = data.build_full_trainset()

algo.fit(trainingSet)

# Make a prediction for user E

prediction = algo.predict('E', 2)
print('\nPrediction for E (item-based): ')
display(prediction.est)

# Configuration to use user-based cosine similarity

sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

trainingSet = data.build_full_trainset()

algo.fit(trainingSet)

# Make a prediction for user E

prediction = algo.predict('E', 2)
print('\nPrediction for E (user-based): ')
display(prediction.est)

# Alternatively, we can predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainingSet.build_anti_testset()
predictions = algo.test(testset)

display(predictions)

## Utilizando el dataset MovieLens 100K

Este ejemplo, sacado de la [documentación de Surprise](https://surprise.readthedocs.io/en/stable/FAQ.html), muestra cómo obtener las top N recomendaciones para cada usuario (i.e. las predicciones más altas para ítems que los usuarios no tenían en sus preferencias) utilizando el [dataset MovieLens 100K](https://grouplens.org/datasets/movielens/100k/).

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])


## Buscando los mejores parámetros de cada algoritmo

La librería surprise incluye una función llamada `GridSearchCV` que evalúa un conjunto de posibles parámetros para un algoritmo y nos devuelve la mejor combinacion de los mismos (en base a una métrica data, como puede ser el RMSE que vimos en clase de teoría).

Utilizando el dataset MovieLens 100K, este primer ejemplo evalúa, para un algoritmo basado en vecinos cercanos, los mejores valores de tres parámetros: métrica de distancia (`name`), el número mínimo de items en común (`min_support`), y se debe aplicar `user-based` o `item-based`.

*Nota*: este ejemplo puede tardar unos cuantos minutos en ejecutarse.

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

En base a estos resultados, vemos que el mejor resultado para este algoritmo se obtiene utilizando un enfoque `item-based`, con un soporte mínimo de 3 y la métrica `msd`.

Evaluamos ahora un enfoque basado en `matrix factorization`, implementado en Surprise en la función `SVD` (entre otras). En este caso, evaluamos tres parámetros: el número de épocas (o iteraciones sobre el dataset completo) del algoritmo de optimización (`n_epochs`), la tasa de aprendizaje (`lr_all`) que indica cómo de grandes deben ser los ajustes en los parámetros (los valores en descomposición en matrices), y `reg_all`, que es una penalización que se añade para evitar el sobre-entrenamiento.

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

En este caso se puede ver que el mejor resultado se obtiene con 10 épocas, una tasa de aprendizaje de 0.005 y una regularización de 0.4.

## Lecturas prácticas sobre collaborative filtering

Al terminar este notebook, se recomienda la lectura de la siguiente entrada de blog en la que se repasan los conceptos básicos del collaborative filtering y se incluyen ejemplos (tomados para este notebook) utilizando Surprise: [Build a Recommendation Engine With Collaborative Filtering](https://realpython.com/build-recommendation-engine-collaborative-filtering/).